In [ ]:
import pandas as pd
import numpy as np
import mta

In [ ]:
def load_mta(start,end):
    '''
    Takes data from the MTA website and loads it into a Pandas DataFrame.  Input
    your required years and date ranges in the download_dates variable and the function will obtain all
    files between those dates and concatenate them into a single dataframe.
    
    Should be string format YYYY-MM-DD
    '''
    
    download_dates = pd.date_range(start=start,end=end,freq='W-SAT')
    download_dates = download_dates.astype(str)
    
    base_url = ("http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt")
    list_of_url = [base_url.format(date[2:4]+date[5:7]+date[8:]) for date in download_dates]
    
    mta = pd.concat((pd.read_csv(url) for url in list_of_url))
    return mta

mta = pd.concat([load_mta('2015-08-01','2015-09-30'),load_mta('2016-08-01','2016-09-30'),
                 load_mta('2017-08-01','2017-09-30'),load_mta('2018-08-01','2018-09-30'),
                 load_mta('2019-08-01','2019-09-30')])

mta.head()          

In [ ]:
import numpy as np

def clean_mta(mta):
    # convert date and time fields into a single datetime field
    mta['datetime'] = mta.DATE + ' ' + mta.TIME  
    mta.datetime = pd.to_datetime(mta.datetime)
    mta = mta.drop(['DATE','TIME'],axis=1)
    
    # get hour and day of week from the datetime field
    mta['hour'] = mta.datetime.dt.hour  
    mta['day_of_week'] = mta.datetime.dt.dayofweek.map(
        {0:'M',1:'Tu',2:'W',3:'Th',4:'F',5:'Sa',6:'Su'})
    
    mta = mta[mta.DESC=='REGULAR'] # eliminate irregular audit data
    
    # sort values by turnstile and datetime so that the riders calculation runs smoothly
    mta = mta.sort_values(['STATION','C/A','UNIT','SCP','datetime']).reset_index(drop=True)
    
    # calculate ridership per time unit based on cumulative values
    # convert all instances of the first time period to null
    mta['riders_in'] = mta.ENTRIES.diff()
    mta['match'] = mta.SCP.eq(mta.SCP.shift())
    mta.loc[mta.match==False,'riders_in'] = np.nan
    return mta

mta = clean_mta(mta)
mta.head()

In [ ]:
mta = mta[(mta.riders_in<20000) & (mta.riders_in>0)]
mta.STATION.nunique()

In [ ]:
mta.loc[mta.STATION=='42 ST-GRD CNTRL','STATION'] = 'GRD CNTRL-42 ST'
mta.loc[mta.STATION=='42 ST-TIMES SQ','STATION'] = 'TIMES SQ-42 ST'
mta.loc[mta.STATION=='59 ST COLUMBUS','STATION'] = '59 ST-COLUMBUS'
mta.loc[mta.STATION=='MAIN ST','STATION'] = 'FLUSHING-MAIN'
mta.loc[mta.STATION=='47-50 STS ROCK','STATION'] = '47-50 ST-ROCK'
mta.loc[mta.STATION=='42 ST-PA BUS TE','STATION'] = '42 ST-PORT AUTH'
mta.loc[mta.STATION=='14 ST-6 AVE','STATION'] = '14 ST'
mta.STATION.nunique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [ ]:
mta['date'] = mta.datetime.dt.date

In [ ]:
by_date = pd.DataFrame(mta.groupby(['STATION','date'])['riders_in'].sum().reset_index())

In [ ]:
by_station = pd.DataFrame(by_date.groupby('STATION').riders_in.mean().sort_values(ascending=False).head(15))

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(y=by_station.index,x='riders_in',data=by_station)
plt.xlabel('Station')
plt.ylabel('Average Daily Passengers')
plt.title('Top 15 Stations by Average Daily Passengers',fontsize=25)
plt.savefig('top15.png',dpi=500);

In [ ]:
by_station.index

In [ ]:
mta = mta.loc[mta['STATION'].isin(by_station.index)]

In [ ]:
reorder_list = ['M','Tu','W','Th','F','Sa','Su']
by_weekday_date = pd.DataFrame(mta.groupby(['day_of_week','date']).riders_in.sum()).reset_index()
by_weekday = pd.DataFrame(by_weekday_date.groupby('day_of_week').riders_in.mean().sort_values(ascending=False).head(20)).reindex(reorder_list)

In [ ]:
by_weekday_date

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=by_weekday.index,y='riders_in',data=by_weekday)
plt.xlabel('Day of Week')
plt.ylabel('Average Daily Riders (millions)')
plt.title('Average Riders by Day of Week, 2015-2019',fontsize=25);

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='day_of_week',y='riders_in',data=by_weekday_date,order=['M','Tu','W','Th','F','Sa','Su'])
plt.xlabel('Day of Week')
plt.ylabel('Number of Daily Passengers (millions)')
plt.title('Distribution of Daily Passengers by Day of Week',fontsize=24)
plt.ylim(0,2000000)
plt.savefig('dayofweek.png',dpi=500);

In [ ]:
mta['time'] = pd.cut(mta.hour,bins=np.linspace(0,24,7))

In [ ]:
by_time_date = pd.DataFrame(mta.groupby(['time','date']).riders_in.sum()).reset_index()
by_time = pd.DataFrame(by_time_date.groupby('time').riders_in.mean().sort_values(ascending=False).head(20))
by_time

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=by_time.index,y='riders_in',data=by_time)
plt.xlabel('Time Range (hour)')
plt.ylabel('Average Passengers')
plt.title('Average Daily Passengers per 4-hour Time Window',fontsize=25)
plt.ticklabel_format(style='plain',axis='y')
plt.savefig('byhour.png',dpi=500);

In [ ]:
by_station

In [ ]:
by_station.to_csv('by_station.csv')